In [51]:
AREAS = [
'Semantic segmentation',
'Image classification',
'Object detection',
'Object Recognition',
'Domain adaptation',
'Image generation',
'Data augmentation (Image augmentation)',
'Pose estimation',
'Autonomous vehicles',
'Denoising',
'Super-Resolution',
'Object Tracking',
'Action Recognition',
'Face Recognition',
'Depth Estimation',
'Optical Character Recognition',
'3D Reconstruction',
'Image Retrieval',
'Optical Flow Estimation',
'Style Transfer',
'Image Compression']

# Google Scholar Crawl

In [69]:
import pandas as pd
from serpapi import GoogleSearch
from tqdm import tqdm

In [70]:
gs_raw = {}
total_paper = 400
for area in tqdm(AREAS):
    area_raw = []
    for ii in range(total_paper//20):
        params = {
        'api_key': '24a5ee79fec15b3de7db738f3b8c3bf6a869c127f2db55f8109723c8007b1798',
        'engine': 'google_scholar',
        'q': 'Semantic Segmentation',
        'hl': 'en',
        'as_ylo': '2000',
        'num': '20',
        'start': f'{20*ii}'
        }
        search = GoogleSearch(params)
        area_raw += search.get_dict().get('organic_results')
    gs_raw[area] = area_raw

100%|██████████| 21/21 [01:24<00:00,  4.05s/it]


In [71]:
gs_results = []
keys_to_extract = ['title', 'link', 'snippet']
for area in AREAS:
    for rank, paper in enumerate(gs_raw[area]):
        paper_result = {'area': area, 'rank': rank + 1}
        for key in keys_to_extract:
            paper_result[key] = paper[key]
        if 'resources' in paper:
            for resource in paper['resources']:
                if resource['file_format'] == 'PDF' and resource['link']:
                    paper_result['pdf_link'] = resource['link']
                    break
        if 'pdf_link' not in paper_result:
            paper_result['pdf_link'] = None
        
        gs_results.append(paper_result)
    
    

In [72]:
gs_df = pd.DataFrame(gs_results)
gs_df.to_csv('gs_results.csv', index=False)

# Semantic Segmenation

In [16]:
import requests


In [105]:
ss_results = []
for area in tqdm(AREAS):
    raw = requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query={}&limit=100&fields=url,title,abstract,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy'.format('+'.join(area.lower().split(' ')))).json().get('data')
    raw += requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query={}&limit=100&offset=100&fields=url,title,abstract,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy'.format('+'.join(area.lower().split(' ')))).json().get('data')
    for rank, paper in enumerate(raw):
        paper['area'] = area
        paper['rank'] = rank + 1
        ss_results.append(paper)

100%|██████████| 21/21 [00:46<00:00,  2.21s/it]


In [112]:
ss_df = pd.DataFrame(ss_results)
ss_df.dropna()
ss_df = ss_df[['area','rank','title','referenceCount', 'url', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'fieldsOfStudy', 'abstract']]

In [113]:
ss_df.to_csv('ss_results.csv', index=False)

# Paperwithcodes

In [33]:
from bs4 import BeautifulSoup

In [65]:
total_paper = 200
pc_results = []
for area in tqdm(AREAS):
    area_formated = '-'.join(area.lower().split(' '))
    for ii in range(total_paper//10):
        html_raw = requests.get(f'https://paperswithcode.com/task/semantic-segmentation?page={ii+1}')
        soup = BeautifulSoup(html_raw.text, 'html.parser')
        paper_cards = soup.find_all('div', class_='paper-card infinite-item')
        for jj, paper_card in enumerate(paper_cards):
            rank = ii * 10 + jj + 1
            result = {'area': area, 'rank': rank}
            result['title'] = paper_card.find('h1').text
            pc_results.append(result)

100%|██████████| 21/21 [06:42<00:00, 19.17s/it]


In [68]:
pc_df = pd.DataFrame(pc_results)
pc_df.to_csv('pc_results.csv', index=False)